# 0. Import Required Libraries

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, models

from collections import OrderedDict

import numpy as np
import mlflow

import scipy.fftpack
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score


from lib.test import *

# 1. Define the Resnet18 Architecture
Only change the last layer to a Fully Connected Layer with a LogSoftmax activation.

In [2]:
model = torchvision.models.resnet18(pretrained=False)
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(in_features=512, out_features=2, bias=True)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
model.fc = classifier

# 2. Load and Preprocess the Data

In [3]:
%%time

# Load
import h5py
from lib.utils import split_trainset

mixdata = h5py.File("../train/scsn_p_2000_2017_6sec_0.5r_pick_train_mix.hdf5", "r")
testdata = h5py.File("../test/scsn_p_2000_2017_6sec_0.5r_pick_test_mix.hdf5", "r")

batch_size = 50
train_size = 1 * 10 ** 4
train_ratio = 0.7
test_size = 1 * 10 ** 4

train_val_data = mixdata["X"][:train_size]
train_val_labels = mixdata["pwave"][:train_size]

trainset, valset = split_trainset(train_val_data, train_val_labels, train_ratio)

Wall time: 315 ms


In [4]:
# Preprocess
from lib.utils import Waveform2Spectrogram, ResnetSpectrogramDataset

t = transforms.Compose([
  Waveform2Spectrogram(),
  transforms.Resize((224, 224)),
  transforms.Grayscale(num_output_channels=3),
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406],
                       [0.229, 0.224, 0.225])
])

trainset = ResnetSpectrogramDataset(trainset, transform=t)
valset = ResnetSpectrogramDataset(valset, transform=t)

In [ ]:
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(valset, batch_size=batch_size,shuffle=True)

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"  

# 3. Add Multi-GPU Support to Model 
In order to run the model on multiple GPU's, we can use the nn.DataParallel layer. This layer requires that we move all tensors to the cuda:0 (the default gpu) before we can pass them through the network. 

In [ ]:
def parallelize(model):
    device_ids = [i for i in range(torch.cuda.device_count())]
    model = torch.nn.DataParallel(model, device_ids=device_ids)
    return model

In [ ]:
#if (device == "cuda"):
#    model = parallelize(model)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

# 4. Define Loss Function and Optimizer
TODO: Describe what loss function and optimizer to use and why.

In [ ]:
from torch import optim

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. Training and Validation

## TODO -- Save model on validation improvement. Then load model when testing and evaluating. 

In [ ]:
mlflow.set_tracking_uri("file:.\mlruns")
mlflow.start_run()

<ActiveRun: info=<RunInfo: run_uuid='52d4999ead034df3a7e4741c9e5f016a', experiment_id=0, name='', source_type=4, source_name='C:\\Users\\rober\\Miniconda3\\envs\\eew\\lib\\site-packages\\ipykernel_launcher.py', entry_point_name=None, user_id='unknown', status=1, start_time=1553739887491, end_time=None, source_version=None, lifecycle_stage='active', artifact_uri='.\\mlruns\\0\\52d4999ead034df3a7e4741c9e5f016a\\artifacts'>, data=None>

In [ ]:
%%time

epochs = 10

train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(epochs):
    model.train()
    train_loss = 0
    for batch, labels in trainloader:
        # ============================================
        #            TRAINING
        # ============================================
        if (device == "cuda"):
            batch, labels = batch.to(device), labels.to(device)
        # Forward pass
        output = model.forward(batch)
        # Clear gradients
        optimizer.zero_grad()
        # Calculate loss
        if (device == "cuda"):
            loss = criterion(output, labels.type(torch.cuda.LongTensor).view(labels.shape, 1))
        else:
            loss = criterion(output, labels.type(torch.LongTensor).view(labels.shape, 1))
        
        train_loss += loss.item()
        # Back propagation
        loss.backward()
        # Update weights
        optimizer.step()
    else:
        with torch.no_grad():
            model.eval()
            val_loss = 0
            
            y_pred = np.array([])
            y_true = np.array([])
            
            for batch, labels in val_loader:
                # ============================================
                #            VALIDATION
                # ============================================
                if (device == "cuda"):
                    batch, labels = batch.to(device), labels.to(device)
                # Forward pass
                log_probs = model.forward(batch)
                probs = torch.exp(log_probs)
                
                top_p, top_class = probs.topk(1, dim=1)
                y_pred = np.append(y_pred, cuda_to_numpy(top_class))
                y_true = np.append(y_true, cuda_to_numpy(labels))
                # Calculate loss
                if (device == "cuda"):
                    loss = criterion(log_probs, labels.type(torch.cuda.LongTensor).view(labels.shape, 1))
                else:
                    loss = criterion(log_probss, labels.type(torch.LongTensor).view(labels.shape, 1))
                val_loss += loss.item()

    # Print epoch summary
    t_loss_avg = train_loss / len(trainloader)
    v_loss_avg = val_loss / len(val_loader)
    accuracy = accuracy_score(y_true, y_pred)

    
    mlflow.log_metric("train_loss", t_loss_avg)
    mlflow.log_metric("val_loss", v_loss_avg)
    mlflow.log_metric("val_accuracy", accuracy)

    
    train_losses.append(t_loss_avg)
    val_losses.append(v_loss_avg)
    val_accuracies.append(accuracy)

    
    print('Epoch [{:5d}/{:5d}] | train loss: {:6.4f} | validation loss: {:6.4f} | validation accuracy: {:6.4f}%'.format(
            epoch+1, epochs, t_loss_avg, v_loss_avg, accuracy * 100))

Epoch [    1/   10] | train loss: 0.4380 | validation loss: 0.5490 | validation accuracy: 76.8000%
Epoch [    2/   10] | train loss: 0.3671 | validation loss: 0.3607 | validation accuracy: 84.6000%


In [ ]:
mlflow.end_run()

## Appendix

### Preparing the Data

One of the challenges of modifying the architecture of an existing CNN is to accomodate for the first layer's input tensor shape.

For this problem specifically, **how do we translate time-series data of shape (300,) into a (224, 224, 3) tensor?**

Briefly, what I did was obtain a spectrogram of the waveform by applying DFT to overlapping windows of the data. This gives us a heatmap of which frequencies had a stronger presence at some specific time. Then, I treat the spectrogram as an image and upscale it to the required dimensions, (224, 224). Finally, I treat the upscaled spectrogram as a greyscale image to give it the 3 channels necessary to work with Resnet.

Below is an elaboration of the process outlined above, with a noise waveform and p-wave waveform used as an example.

In [ ]:
"""
First, let's load the data and compare the two types of waveforms.
"""
file = h5py.File(train_filepath, 'r')
dataset = file.get(features_key)[:dataset_size]
file.close()

noise = dataset[0]
pwave = dataset[1]

plt.title('Noise(R) vs P-wave(B)')
plt.plot(range(len(noise)), noise, 'r.-')
plt.plot(range(len(pwave)), pwave, 'b.-')
plt.show()

In [ ]:
"""
Then, let's take a look at the Fast Fourier Transform (just the real part) of the waveforms.
We actually don't need this to calculate the spectrograms, but it's here to get a better
understanding of how the waveforms can be interpreted.
"""
noise_fft = list(map(lambda x: 0 if x < 0 else x, scipy.fftpack.rfft(noise)))
pwave_fft = list(map(lambda x: 0 if x < 0 else x, scipy.fftpack.rfft(pwave)))
t = np.linspace(0, 3, 300)

plt.title("FFT Noise")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.plot(t, noise_fft)
plt.show()

plt.title("FFT P-Wave")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.plot(t, pwave_fft)
plt.show()

In [ ]:
"""
Next, let's get the spectrograms using scipy.signal.spectrogram
"""
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html
from scipy.signal import spectrogram

noise_freqs, noise_times, noise_Sx = spectrogram(noise, fs=100, window='hamming',
                                                nperseg=30, noverlap=0,
                                                detrend='linear', scaling='spectrum')
pwave_freqs, pwave_times, pwave_Sx = spectrogram(pwave, fs=100, window='hamming',
                                                nperseg=30, noverlap=0,
                                                detrend='linear', scaling='spectrum')

plt.pcolormesh(noise_times, noise_freqs, 10 * np.log10(noise_Sx), cmap='viridis')
plt.title('Noise Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]');
plt.show()

plt.title('P-Wave Spectrogram')
plt.pcolormesh(pwave_times, pwave_freqs, 10 * np.log10(pwave_Sx), cmap='viridis')
plt.title('P-Wave Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.show()

In [ ]:
"""
We can already see how Resnet can accept these spectograms as images, but the
spectograms shape isn't quite right. Right now, it's (16, 10) and is determined
by the arguments passed to scipy.signal.spectrogram. Either way, we can resize
it to (224, 224) using torchvision.transforms.Grayscale (skimage.transform.resize works too).
"""
# https://scikit-image.org/docs/dev/api/skimage.transform.html?highlight=resize#skimage.transform.resize
from skimage.transform import resize

noise_Sx = resize(noise_Sx, (224, 224), mode='reflect')
pwave_Sx = resize(pwave_Sx, (224, 224), mode='reflect')

# In order to correctly print the spectrogram, we must also change the time and
# frequency lists to reflect the resized spectrograms.
noise_times = np.linspace(noise_times[0], noise_times[-1], 224)
noise_freqs = np.linspace(noise_freqs[0], noise_freqs[-1], 224)
pwave_times = np.linspace(pwave_times[0], pwave_times[-1], 224)
pwave_freqs = np.linspace(pwave_freqs[0], pwave_freqs[-1], 224)

plt.pcolormesh(noise_times, noise_freqs, 10 * np.log10(noise_Sx), cmap='viridis')
plt.title('Noise Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]');
plt.show()

plt.title('P-Wave Spectrogram')
plt.pcolormesh(pwave_times, pwave_freqs, 10 * np.log10(pwave_Sx), cmap='viridis')
plt.title('P-Wave Spectrogram')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [s]')
plt.show()

In [ ]:
"""
Getting closer now! The spectogram's shapes are (224, 224), but we're still missing
the three channels (RGB). So, let's treat the spectrograms like greyscale images to derive
the other channels. We can do this using skimage.color.gray2rgb (either spelling of gray works!).
"""
# http://scikit-image.org/docs/dev/user_guide/viewer.html
from skimage.color import gray2rgb
noise_Sx = gray2rgb(noise_Sx)
pwave_Sx = gray2rgb(pwave_Sx)
print(f"Noise Spectrogram Shape: {noise_Sx.shape}\nP-Wave Spectrogram Shape: {pwave_Sx.shape}")

In [ ]:
# def imshow(inp, title=None):
#     """Imshow for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated


# # Get a batch of training data
# sample = next(iter(train_dataloader))

# # Make a grid from batch
# out = torchvision.utils.make_grid(sample["waveform"])

# imshow(out)

## References